# Clase 18: Preparación de datos para ML - Limpieza y transformación con SQL y Snowpark

In [ ]:
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
session = get_active_session()

In [ ]:
# 2. Seleccionar las tablas necesarias (ORDERS, CUSTOMER, NATION, REGION)
orders_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS")
customer_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER")
nation_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION")
region_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.REGION")

In [ ]:
# 3. Filtrar datos para Estados Unidos (USA)
usa_region_key = region_table.filter(F.col("R_NAME") == 'AMERICA').select("R_REGIONKEY")
usa_nation_key = nation_table.filter( (F.col("N_NAME") == 'UNITED STATES') & (F.col("N_REGIONKEY").isin(usa_region_key))).select("N_NATIONKEY")
usa_customer_keys = customer_table.filter(F.col("C_NATIONKEY").isin(usa_nation_key)).select("C_CUSTKEY")
usa_orders = orders_table.filter(F.col("O_CUSTKEY").isin(usa_customer_keys))

In [ ]:
# 4. Agrupar las ventas por fecha para obtener las ventas diarias totales
daily_sales_spdf = usa_orders.group_by(
    F.to_date(F.col("O_ORDERDATE")).alias("ORDER_DATE")
    ).agg(F.sum("O_TOTALPRICE").alias("TOTAL_SALES"))

In [ ]:
daily_sales_spdf.count()

In [ ]:
# 5. Convertir a Pandas DataFrame para visualización y ML con scikit-learn
daily_sales_pdf = daily_sales_spdf.to_pandas()

In [ ]:
# 6. Ordenar por fecha para la visualización de series temporales
daily_sales_pdf = daily_sales_pdf.sort_values(by='ORDER_DATE')

In [ ]:
# 7. Visualizar las ventas diarias totales usando Plotly
px.line(daily_sales_pdf, x="ORDER_DATE", y="TOTAL_SALES",
              title='Ventas Diarias Totales en Estados Unidos', render_mode='svg')

# Clase 19: Integración con herramientas de ML - Conexión con Scikit-learn y Entrenamiento

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# 8. Preparar los datos para Scikit-learn
daily_sales_pdf['ORDER_DATE_NUM'] = pd.to_datetime(daily_sales_pdf['ORDER_DATE']).astype('int64') // 10**9 # Convertir fechas a numérico (timestamp segundos)
X = daily_sales_pdf[['ORDER_DATE_NUM']] # Característica: Fecha numérica
y = daily_sales_pdf['TOTAL_SALES']      # Variable objetivo: Ventas totales

In [ ]:
# 9. Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 10. Entrenar el modelo de Regresión Lineal con Scikit-learn
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# 11. Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [ ]:
# 12. Evaluar el modelo (Error Cuadrático Medio)
mse = mean_squared_error(y_test, y_pred)
print(f'Error Cuadrático Medio en el conjunto de prueba: {mse}')

In [ ]:
# 13. Visualizar las predicciones vs. valores reales en el conjunto de prueba
predictions_df = pd.DataFrame({'Fecha': daily_sales_pdf['ORDER_DATE'].iloc[X_test.index].values, # Obtener fechas correspondientes a X_test
                                 'Ventas Reales': y_test.values,
                                 'Ventas Predichas': y_pred})

In [ ]:
predictions_df = predictions_df.groupby('Fecha').agg({
    'Ventas Reales':'sum',
    'Ventas Predichas':'sum'
}).reset_index()

In [ ]:
px.line(predictions_df, x='Fecha', y=['Ventas Reales', 'Ventas Predichas'],
                           title='Predicciones de Ventas vs. Ventas Reales (Conjunto de Prueba)', render_mode='svg')

# Clase 20: Ejecución de scripts de ML - Guardar y Predecir con el Modelo

In [ ]:
import joblib
import os

In [ ]:
CREATE STAGE my_ml_models_stage;

In [ ]:
# 14. Guardar el modelo entrenado en stage de Snowflake
model_stage_path = "@my_ml_models_stage/linear_regression_sales_model.joblib" # Reemplaza 'my_ml_models_stage' con el nombre de tu stage
joblib.dump(model, 'model.joblib') # Guarda localmente en la sesión de Snowsight (memoria)
session.file.put("model.joblib", model_stage_path, overwrite=True) # Sube al stage


In [ ]:
# 15. Cargar el modelo desde el stage de Snowflake
downloaded_files = session.file.get(model_stage_path, './')
loaded_model = joblib.load(downloaded_files[0].file) # Accedemos al atributo 'file' del GetResult

In [ ]:
# 16. Predecir nuevas ventas para un rango de fechas futuras (ejemplo: 3 meses después del último dato)
last_date_numeric = X['ORDER_DATE_NUM'].max() # Fecha numérica más reciente en los datos
future_dates_numeric = [last_date_numeric + (i * 86400) for i in range(1, 91)] # 90 días en segundos
future_dates_dt = pd.to_datetime(future_dates_numeric, unit='s') # Convertir a datetime
future_dates_df = pd.DataFrame({'ORDER_DATE_NUM': future_dates_numeric, 'ORDER_DATE': future_dates_dt})

future_predictions = loaded_model.predict(future_dates_df[['ORDER_DATE_NUM']])

In [ ]:
# 17. Crear DataFrame con predicciones futuras para visualización
future_predictions_df = pd.DataFrame({'Fecha Futura': future_dates_df['ORDER_DATE'],
                                       'Ventas Predichas': future_predictions})

In [ ]:
# 18. Visualizar las predicciones futuras junto con las ventas históricas
fig_future_predictions = px.line(future_predictions_df, x='Fecha Futura', y='Ventas Predichas',
                                   title='Predicciones de Ventas Futuras (3 Meses)', render_mode='svg')

fig_future_predictions